SQLite 是一种嵌入式数据库，它的数据库是一个文件。使用 Python 操作 SQLite 数据库的基本流程如下：

- 通过 `sqlite3.open()` 创建于数据路的连接对象 connectoin;
- 通过 `connection.cursor()` 创建光标对象 cursor;
- 通过 `cursor.execute()` 执行 SQL 语句；
- 通过 `connection.commit()` 提交当前的事务，或者通过 `cursor.fetchall()` 获得查询结果；
- 通过 `connection.close()` 关闭于数据库文件的连接。

In [2]:
import sqlite3

conn = sqlite3.connect("test.db")
c = conn.cursor()

c.execute("CREATE TABLE IF NOT EXISTS students (sid INTEGER PRIMARY KEY, name TEXT)")
conn.commit()

conn.close()

这里在当前工作目录下创建了一个数据库文件 `test.db`，另外还可以使用`":memory:"` 建立内存数据库。

> 内存数据库是一种将全部内容存放在内存中，而非传统数据库那样存放在外部存储器中的数据库。这种数据库的读写性能很高，主要用于在对性能要求极高的环境中，但是在服务器关闭后会立刻丢失全部存储的数据。

In [4]:
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute("CREATE TABLE students (sid INTEGER PRIMARY KEY, name TEXT)")
conn.commit()

c.execute("INSERT INTO students VALUES(?, ?)", (1, "Alice"))
c.execute("INSERT INTO students VALUES(?, ?)", (2, "Bob"))
c.execute("INSERT INTO students VALUES(?, ?)", (3, "Peter"))

c.execute("DELETE FROM students WHERE sid = ?", (1,))
c.execute("UPDATE students SET name = ? WHERE sid = ?", ("Mark", 3))

conn.commit()

c.execute("SELECT * FROM students")
print(c.fetchall())

conn.close()

[(2, 'Bob'), (3, 'Mark')]


`fetchall` 返回满足所有要求的 records 列表。

## 封装成 Python 函数

In [5]:
import sqlite3
import os, sys

def initialize(conn):
    c = conn.cursor()
    c.execute("CREATE TABLE students (sid INTEGER PRIMARY KEY, name TEXT)")
    conn.commit()

def insert(conn, sid, name):
    c = conn.cursor()
    t = (sid, name)
    c.execute("INSERT INTO students VALUES (?, ?)", t)
    conn.commit()
    
def delete(conn, sid):
    c = conn.cursor()
    t = (sid, )
    c.execute("DELETE FROM students WHERE sid = ?", t)
    conn.commit()
    
def update(conn, sid, name):
    c = conn.cursor()
    t = (name, sid)
    c.execute("UPDATE students SET name = ? WHERE sid = ?", t)
    conn.commit()
    
def display(conn):
    c = conn.cursor()
    c.execute("SELECT * FROM students");
    print(c.fetchall())
    
db_name = ":memory:"
conn = sqlite3.connect(db_name)

initialize(conn)

print("Insert 3 records.")
insert(conn, 1, "Alice")
insert(conn, 2, "Bob")
insert(conn, 3, "Peter")
display(conn)

print("Delete the record where sid = 1.")
delete(conn, 1)
display(conn)

print("Update the record where sid = 3.")
update(conn, 3, "Mark")
display(conn)

conn.close()

Insert 3 records.
[(1, 'Alice'), (2, 'Bob'), (3, 'Peter')]
Delete the record where sid = 1.
[(2, 'Bob'), (3, 'Peter')]
Update the record where sid = 3.
[(2, 'Bob'), (3, 'Mark')]
